In [1]:
!pip install tensorflow

In [2]:
!pip install scikeras

In [1]:
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from scikeras.wrappers import KerasClassifier
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
df = pd.read_csv("data/points_data.csv")

In [3]:
# Select quantitatives variables
cuant_var = df[["altitud", "pendiente", "orientacion", "t_max", "u", "v", "specific_humidity", "relative_humidity"]]
# Normalize
mean = cuant_var.mean()
mean.to_csv("data/means.csv")
std = cuant_var.std()
std.to_csv("data/stds.csv")
normalized_df = (cuant_var-cuant_var.mean())/cuant_var.std()

In [4]:
# Split dataset
y = df["is_fire"]
xTrain, xTest, yTrain, yTest = train_test_split(normalized_df, y, test_size=0.2, random_state=12)

In [7]:
# Create function returning a model
def design_model (dropout_rate, weight_constraint):

    model = keras.Sequential()

    model.add(keras.layers.Dense(64, input_shape=(8,), activation='relu', kernel_constraint=keras.constraints.MaxNorm(weight_constraint)))
    model.add(keras.layers.Dropout(dropout_rate))

    model.add(keras.layers.Dense(128, activation='relu', kernel_constraint=keras.constraints.MaxNorm(weight_constraint)))
    model.add(keras.layers.Dropout(dropout_rate))

    model.add(keras.layers.Dense(256, activation='relu', kernel_constraint=keras.constraints.MaxNorm(weight_constraint)))
    model.add(keras.layers.Dropout(dropout_rate))

    model.add(keras.layers.Dense(128, activation='relu', kernel_constraint=keras.constraints.MaxNorm(weight_constraint)))
    model.add(keras.layers.Dropout(dropout_rate))

    model.add(keras.layers.Dense(64, activation='relu', kernel_constraint=keras.constraints.MaxNorm(weight_constraint)))
    model.add(keras.layers.Dropout(dropout_rate))
    
    model.add(keras.layers.Dense(2, activation='softmax'))
    
    model.compile(loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    return model

model = KerasClassifier(model=design_model, verbose=0)

In [8]:
# Define the search parameters
epochs = [10,20,50,100]
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
weight_constraint = [1.0, 2.0, 3.0, 4.0, 5.0]
dropout_rate = [ 0.2, 0.3, 0.4, 0.5]

In [10]:
param_grid = dict(model__dropout_rate=dropout_rate, model__weight_constraint=weight_constraint, optimizer=optimizer, epochs=epochs)

# grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)

In [11]:
grid_result = grid.fit(xTrain, yTrain)

PicklingError: Could not pickle the task to send it to the workers.

In [ ]:
# Print results
print(f'Best:{grid_result.best_score_} using {grid_result.best_params_}')

for mean, stdev, param in zip(grid_result.cv_results_['mean_test_score'], 
                              grid_result.cv_results_['std_test_score'], grid_result.cv_results_['params']):
    print(f"{mean} {stdev} with: {param}")

In [ ]:
##### Design final model with best parameters
model = keras.Sequential()

model.add(keras.layers.Dense(64, input_shape=(19,), activation='relu', kernel_constraint=keras.constraints.MaxNorm(1)))
model.add(keras.layers.Dropout(0.3))

model.add(keras.layers.Dense(128, activation='relu', kernel_constraint=keras.constraints.MaxNorm(1)))
model.add(keras.layers.Dropout(0.3))

model.add(keras.layers.Dense(256, activation='relu', kernel_constraint=keras.constraints.MaxNorm(1)))
model.add(keras.layers.Dropout(0.3))

model.add(keras.layers.Dense(128, activation='relu', kernel_constraint=keras.constraints.MaxNorm(1)))
model.add(keras.layers.Dropout(0.3))

model.add(keras.layers.Dense(64, activation='relu', kernel_constraint=keras.constraints.MaxNorm(1)))
model.add(keras.layers.Dropout(0.3))
    
model.add(keras.layers.Dense(2, activation='softmax'))

In [ ]:
# Define the accuracy metrics and parameters
model.compile(optimizer="Adamax", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Run the model
model.fit(xTrain, yTrain, epochs=50)

In [ ]:
# Export model
model.save("data/nn_fire_risk.h5")

In [6]:
# Load model
model = keras.models.load_model('data/nn_fire_risk.h5')

In [7]:
# Predict for test data 
yTestPredicted = model.predict(xTest)
yTestPredicted = yTestPredicted[:,1]

# Calculate the error metrics
yTestPredicted = (yTestPredicted>0.5).astype(int)
cMatrix = confusion_matrix(yTest, yTestPredicted)
pScore = precision_score(yTest, yTestPredicted)
rScore = recall_score(yTest, yTestPredicted)

print("\nConfusion matrix:", cMatrix)
print("\nP-Score: %.3f, R-Score: %.3f" % (pScore, rScore))

7/7 [==============================] - 0s 661us/step

Confusion matrix: [[168   2]
 [  6  24]]

P-Score: 0.923, R-Score: 0.800


In [9]:
# Evaluate the model on test data
loss, accuracy = model.evaluate(xTest, yTest)

# Print the evaluation results
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

7/7 [==============================] - 0s 922us/step - loss: 0.1359 - accuracy: 0.9600
Test Loss: 0.1358533501625061
Test Accuracy: 0.9599999785423279
